In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=5
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, transient=True)
solverS.output_file = 'solid_outputs/resFDSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)
tempF = InterfaceData(size=inodes, value=300.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FDSR%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FDSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve the wall temperature for convergence testing
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
#         if pc_counts == 1 or pc_counts == 2:
#             print(abTempF.curr)
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid flux interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        TempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
#         if pc_counts == 1 or pc_counts == 2:
#             print(abTempS.curr)
        
        # mapping
        tempF.curr[:] = TempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(tempF)
            under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=38.
step=1, pc_iterations=27.
step=2, pc_iterations=20.
step=3, pc_iterations=16.
step=4, pc_iterations=20.
step=5, pc_iterations=21.
step=6, pc_iterations=21.
step=7, pc_iterations=21.
step=8, pc_iterations=21.
step=9, pc_iterations=21.
step=10, pc_iterations=21.
step=11, pc_iterations=20.
step=12, pc_iterations=20.
step=13, pc_iterations=20.
step=14, pc_iterations=19.
step=15, pc_iterations=19.
step=16, pc_iterations=19.
step=17, pc_iterations=19.
step=18, pc_iterations=18.
step=19, pc_iterations=18.
step=20, pc_iterations=18.
step=21, pc_iterations=18.
step=22, pc_iterations=18.
step=23, pc_iterations=17.
step=24, pc_iterations=17.
step=25, pc_iterations=17.
step=26, pc_iterations=17.
step=27, pc_iterations=17.
step=28, pc_iterations=16.
step=29, pc_iterations=16.
step=30, pc_iterations=16.
step=31, pc_iterations=16.
step=32, pc_iterations=16.
step=33, pc_iterations=16.
step=34, pc_iterations=15.
step=35, pc_iterations=15.
step=36, pc_iterations=15.
step=37, pc

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFDSR%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.70798834 0.71278489 0.72005436 0.72685389 0.73309629 0.73857156
 0.74357595 0.74810032 0.75224987 0.75617534 0.75975973 0.76312022
 0.76623377 0.76916569 0.77188948 0.77445325 0.77684619 0.77910892
 0.7812268  0.7832095  0.7850751  0.78682287 0.78847164 0.79001289
 0.79145757 0.79280976 0.79408524 0.7952765  0.79639679 0.7974442
 0.79842137 0.79933862 0.80019342 0.80099067 0.80173947 0.80243281
 0.80307915 0.80367752 0.80423692 0.80475285 0.80523062 0.80567347
 0.80607887 0.80645127 0.80679188 0.80710174 0.80738278 0.80763546
 0.80786086 0.80806211 0.80823577 0.80838651 0.80851212 0.8086144
 0.80869441 0.80874973 0.808785   0.80879573 0.80878456 0.80875017
 0.80869249 0.80861315 0.80850754 0.80837877 0.80822206 0.80804047
 0.80783192 0.80759222 0.80732443 0.80702321 0.80668736 0.80631786
 0.80590748 0.80546004 0.80496635 0.80442755 0.80383389 0.80319026
 0.80248812 0.80171917 0.8008857  0.79997152 0.79897645 0.79789874
 0.79671549 0.79542842 0.79401445 0.79247242 0.79078104 0.788912